<a href="https://colab.research.google.com/github/Silver15987/facemask-detection/blob/main/yolov5-optimized-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()

YOLOv5 🚀 v6.2-195-gdf80e7c Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)


Setup complete ✅ (12 CPUs, 83.5 GB RAM, 38.8/166.8 GB disk)


In [ ]:
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.12.1+cu113 _CudaDeviceProperties(name='A100-SXM4-40GB', major=8, minor=0, total_memory=40536MB, multi_processor_count=108)


In [ ]:
import xml.etree.ElementTree as ET
import glob
import os
import json

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [ ]:
import json 
import os
from pathlib import Path

# your api key
api_key = {"username":"pratimanjoshi","key":"4749138ea73272a7a8b9b58f44a06211"}

# uses pathlib Path
kaggle_path = Path('/root/.kaggle')
os.makedirs(kaggle_path, exist_ok=True)

# opens file and dumps python dict to json object 
with open (kaggle_path/'kaggle.json', 'w') as handl:
    json.dump(api_key,handl)

os.chmod(kaggle_path/'kaggle.json', 600)  

In [ ]:
!kaggle datasets download andrewmvd/face-mask-detection -p /content/datasets/ --unzip

 98% 391M/398M [00:01<00:00, 251MB/s]
100% 398M/398M [00:01<00:00, 245MB/s]


In [ ]:
classes = []
input_dir = "/content/datasets/annotations"
output_dir = "/content/datasets/labels/"
image_dir = "/content/datasets/images"
os.mkdir(output_dir)

In [ ]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

In [ ]:
files = glob.glob(os.path.join(input_dir, '*.xml'))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue
    
    result = []
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    
    for obj in root.findall('object'):
        label = obj.find("name").text
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")
        
    if result:
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

In [ ]:
# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

Train test split and preperation of data

In [ ]:
import os
import numpy as np

from tqdm.notebook import tqdm
import shutil

In [ ]:
root_dir = "/content/dataset/"
new_labels_dir = "/content/datasets/labels/"
images_dir = "/content/datasets/images/"

if not os.path.exists(root_dir):
  os.mkdir(root_dir)

for i in ["train", "val","test"]:
    if not os.path.exists(root_dir+i):
      os.mkdir(root_dir+i)
    
  
images_list = list(sorted(os.listdir(images_dir)))
new_labels_list = list(sorted(os.listdir(new_labels_dir)))

len(images_list)

853

In [ ]:
len(images_list)*0.7 # 70% => 598
len(images_list)*0.15
599 +127 +127

853

In [ ]:
np.random.seed(45)
np.random.shuffle(images_list)
np.random.seed(45)
np.random.shuffle(new_labels_list)

In [ ]:
train_or_test = 'train/'
for i in tqdm(range(len(images_list))):
  # root_dir + train +images
    if i==599 :
        #print("Test")
        train_or_test = 'val/'
        print(root_dir+train_or_test)

    if i==599+127 :
        #print("Test")
        train_or_test = 'test/'
        print(root_dir+train_or_test)
  
    path = root_dir+train_or_test
    
    #image and label in the same folder
    shutil.copy(images_dir+images_list[i], path) 
    shutil.copy(new_labels_dir+new_labels_list[i], path)

  0%|          | 0/853 [00:00<?, ?it/s]

/content/dataset/val/
/content/dataset/test/


In [ ]:
len(os.listdir("/content/dataset/test"))

254

In [ ]:
text_to_write = "train: /content/dataset/train\n"
text_to_write += "val: /content/dataset/val\n"
text_to_write += "test: /content/dataset/test\n\n"
text_to_write += "# number of classes\nnc: 3\n\n"
text_to_write += "# class names\nnames: [\"with_mask\", \"mask_weared_incorrect\", \"without_mask\"]\n"

In [ ]:
with open("/content/data_face_mask2.yaml",'w') as f:
    print(f.write(text_to_write))

188


In [ ]:
with open("/content/data_face_mask2.yaml") as f:
    print(f.read())
shutil.copy("/content/data_face_mask2.yaml", root_dir)

train: /content/dataset/train
val: /content/dataset/val
test: /content/dataset/test

# number of classes
nc: 3

# class names
names: ["with_mask", "mask_weared_incorrect", "without_mask"]



'/content/dataset/data_face_mask2.yaml'

In [ ]:
import torch
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.12.1+cu113 _CudaDeviceProperties(name='A100-SXM4-40GB', major=8, minor=0, total_memory=40536MB, multi_processor_count=108)


In [ ]:
%cat {root_dir}data_face_mask2.yaml
data_dir = '/content/datasets/'

train: /content/dataset/train
val: /content/dataset/val
test: /content/dataset/test

# number of classes
nc: 3

# class names
names: ["with_mask", "mask_weared_incorrect", "without_mask"]


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.6 MB/s 
     |████████████████████████████████| 182 kB 75.6 MB/s 
     |████████████████████████████████| 162 kB 87.4 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 162 kB 91.6 MB/s 
     |████████████████████████████████| 158 kB 90.3 MB/s 
     |████████████████████████████████| 157 kB 90.4 MB/s 
     |████████████████████████████████| 157 kB 91.1 MB/s 
     |████████████████████████████████| 157 kB 97.2 MB/s 
     |████████████████████████████████| 157 kB 94.1 MB/s 
     |████████████████████████████████| 157 kB 97.0 MB/s 
     |████████████████████████████████| 157 kB 98.4 MB/s 
     |████████████████████████████████| 157 kB 98.8 MB/s 
     |████████████████████████████████| 156 kB 93.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=ad

In [ ]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
project='face_mask'
name='run_optimized'

In [ ]:
!python train.py --img 640 --batch 16 --epochs 200 --data /content/dataset/data_face_mask2.yaml --weights yolov5m.pt --cache --project face_mask --name run_optimized

wandb: Currently logged in as: silver15987 (amos-tech). Use `wandb login --relogin` to force relogin
train: weights=yolov5m.pt, cfg=, data=/content/dataset/data_face_mask2.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=face_mask, name=run_optimized, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-195-gdf80e7c Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warm

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path = '/content/yolov5/face_mask/run_optimized/weights/best.pt', force_reload=True)

In [ ]:
video_path = "kaggle\CV\Mask_detection\mask_movie.mp4"

cap = cv2.VideoCapture(0)

assert cap.isOpened()
while(cap.isOpened()):
    ret, frame = cap.read()
    assert not isinstance(frame,type(None)), 'frame not found'
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
%pylab inline
test_photo_path = Path(".jpg")

results = model(test_photo_path)
plt.imshow(np.squeeze(results.render()))
plt.axis(False)
plt.show()

test_photo_path = Path(".jpg")

results = model(test_photo_path)
plt.imshow(np.squeeze(results.render()))
plt.axis(False)
plt.show()


In [ ]:
!python val.py --weights "/content/yolov5/face_mask/run_optimized/weights/best.pt" --data "/content/dataset/data_face_mask2.yaml" --img 640 --iou 0.65 --task test

val: data=/content/dataset/data_face_mask2.yaml, weights=['/content/yolov5/face_mask/run_optimized/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.2-195-gdf80e7c Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 212 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs
test: Scanning '/content/dataset/test' images and labels...127 found, 0 missing, 0 empty, 0 corrupt: 100% 127/127 [00:00<00:00, 3193.43it/s]
test: New cache created: /content/dataset/test.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 4/4 [00:04<00:00,  1.22s/it]
                   all        127        582       0.88      0.738      0.803      0.533
  

In [ ]:
!cp -r "/content/yolov5/runs" "/content/drive/MyDrive/backup"

In [ ]:
!python export.py --weights "/content/yolov5/face_mask/run_optimized/weights/best.pt" --include torchscript

Traceback (most recent call last):
  File "export.py", line 70, in <module>
    from models.yolo import ClassificationModel, Detect, DetectionModel, SegmentationModel
ImportError: cannot import name 'ClassificationModel' from 'models.yolo' (/content/yolov5/models/yolo.py)
